<h1> Comparación de Modelos

<h3> Creación de gráficos y GIFs para comparar entre ambos tipos de modelos.

Equipo de Reto 5

Inteligencia artificial avanzada para la ciencia de datos II (Grupo 502)

30 de noviembre de 2023

### Preparar el entorno de trabajo

In [ ]:
# Importar librerias y módulos necesarios
from keras.models import load_model
import os
import numpy as np
import random
import cv2
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import random
import imageio

In [ ]:
# Definir el directorio de trabajo actual
os.chdir('/content/drive/Shareddrives/Penta Tech/Reto')

In [ ]:
# Definir el directorio con los mejores modelos
models_directory = os.path.join(os.getcwd(), 'Models')

In [ ]:
# Definir el directorio para guardar los GIFs generados
gifs_directory = os.path.join(os.getcwd(), 'GIFs')
os.mkdir(gifs_directory)

In [ ]:
# Definir dimensiones importantes de los datos
img_height = 112
img_width = 112
img_channels = 1
mask_channels = 1
landmarks_channels = 7

In [ ]:
# Cargar los mejores modelos
mask_model = load_model(os.path.join(models_directory, 'masks_08_0.99.h5'), compile=False)
landmarks_model = load_model(os.path.join(models_directory, 'landmarks_05_0.45.h5'), compile=False)

### Calcular el coeficiente de Sorensen-Dice para todos los datos del conjunto de prueba



In [ ]:
# Definir el directorio con los conjuntos de datos
data_directory = os.path.join(os.getcwd(), 'CompleteDatasets')

In [ ]:
# Cargar el conjunto de prueba
test_images = np.load('/content/drive/Shareddrives/Penta Tech/Reto/CompleteDatasets/test_images.npy')
test_masks = np.load('/content/drive/Shareddrives/Penta Tech/Reto/CompleteDatasets/test_masks.npy')
print('Dimensiones del conjunto de prueba:', test_images.shape, test_masks.shape)

In [ ]:
# # Predecir con ambos modelos todos los datos en el conjunto de prueba
mask_predictions = mask_model.predict(test_images)
landmark_predictions = landmarks_model.predict(test_images)

In [ ]:
# Crear listas para guardar los coeficientes
dice_scores_masks = []
dice_scores_landmarks = []

# Para cada predicción del conjunto de prueba
for prediction in range(len(mask_predictions)):

  # Seleccionar las predicciones correspondiente
  true_prediction = test_masks[prediction]
  mask_prediction = (mask_predictions[prediction] > 0.5)
  landmark_prediction = landmark_predictions[prediction]

  # Generar la máscara de la predicción basada en puntos de interés
  coords = []
  for channel in range(landmark_prediction.shape[2]):
    landmark = (landmark_prediction[:,:,channel] >= np.max(landmark_prediction[:,:,channel]))
    landmark = np.unravel_index(np.argmax(landmark), landmark.shape)
    landmark = (landmark[1], landmark[0])
    if landmark != (0, 0):
      coords.append(landmark)
  mask = np.zeros_like(mask_prediction).astype(np.uint8)
  coords = np.array(coords).astype(np.int32).reshape((-1, 1, 2))
  cv2.fillPoly(mask, [cv2.convexHull(coords)], (255))
  landmark_prediction = (mask / 255.0).astype(np.float16)

  # Añadir los coeficientes a su lista correspondiente
  mask_intersection = np.sum((true_prediction == 1) & (mask_prediction == 1))
  mask_total = np.sum(true_prediction == 1) + np.sum(mask_prediction == 1)
  landmark_intersection = np.sum((true_prediction == 1) & (landmark_prediction == 1))
  landmark_total = np.sum(true_prediction == 1) + np.sum(landmark_prediction == 1)
  dice_scores_masks.append(float(((2*mask_intersection)/mask_total)*100))
  dice_scores_landmarks.append(float(((2*landmark_intersection)/landmark_total)*100))

In [ ]:
# Crear un histograma de con todos los coeficientes del conjunto de prueba
plt.figure(figsize=(7, 4))
plt.hist(dice_scores_masks, bins=np.arange(min(dice_scores_masks), max(dice_scores_masks) + 0.5, 0.5), alpha=0.6, label=f'Mask Model (Avg: {statistics.mean(dice_scores_masks):.3f})')
plt.hist(dice_scores_landmarks, bins=np.arange(min(dice_scores_landmarks), max(dice_scores_landmarks) + 0.5, 0.5), alpha=0.6, label=f'Landmark Model (Avg: {statistics.mean(dice_scores_landmarks):.3f})')
plt.title('Dice Score Distribution')
plt.xlabel('Dice Score')
plt.ylabel('Frequency')
plt.grid(alpha = 0.5)
plt.legend()

### Crear GIFs con las máscaras generadas por cada modelo

In [ ]:
# Crear una lista con videos que pertenecen al conjunto de datos externos
apical_files = os.listdir(os.path.join(os.getcwd(), 'Apical'))

In [ ]:
# Crear una lista con videos que pertenecen al conjunto de datos de prueba
test_files = pd.read_csv(os.path.join(os.getcwd(), 'EchoNet-Dynamic/FileList.csv'))
test_files.drop(columns=['EF', 'ESV', 'EDV', 'FrameHeight', 'FrameWidth', 'FPS', 'NumberOfFrames'], inplace=True)
test_files = list(test_files[test_files['Split'] == 'TEST'].sample(n=len(apical_files)).FileName.values)

In [ ]:
# Combinar diez elementos aleatorios de las listas con datos externos y de prueba
test_files = random.sample(apical_files, 5) + random.sample(test_files, 5)

# Para diez videos aleatorios en la lista
for video in test_files:

  # Definir la ubicación del video dependiendo del conjunto al que pertenece
  if len(video) == 9:
    path = os.path.join(os.getcwd(), 'Apical', video)
  else:
    video = video + '.avi'
    path = os.path.join(os.getcwd(), 'EchoNet-Dynamic/Videos', video)

  # Guardar todos los fotogramas del video en un arreglo
  cap = cv2.VideoCapture(path)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  frames = np.empty((0, img_height, img_width, img_channels), dtype=np.float16)
  for f in range(frame_count):
    cap.set(cv2.CAP_PROP_POS_FRAMES, f)
    _, frame = cap.read()
    frame = (cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    if frame.shape != (img_height, img_width):
      frame = cv2.resize(frame, (img_height, img_height))
    frame = (frame / 255.0).astype(np.float16)
    frames = np.append(frames, [np.expand_dims(frame, axis=-1)], axis=0)
  cap.release()

  # Imprimir la información del video
  print(f'VIDEO: {video[:-4]}.gif  FOTOGRAMAS: {frame_count}  FPS: {fps}')

  # Generar predicciones para todos los fotogramas del video con ambos modelo
  mask_predictions = mask_model.predict(frames)
  landmark_predictions = landmarks_model.predict(frames)

  # Guardar en una lista todos los fotogramas con la máscaras
  mask_frames = []
  for frame_number in range(frame_count):
    frame = (frames[frame_number] * 255).astype(np.uint8)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    mask = np.squeeze((mask_predictions[frame_number] > 0.5) * 255).astype(np.uint8)
    frame[:,:,2] = cv2.addWeighted(mask, 0.75, frame[:,:,2], 1, 0)
    cv2.putText(frame, ('M1 ' + video[:-4]), (1,img_height-3), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (255, 255, 255), 1)
    mask_frames.append(frame)

  # Guardar en una lista todos los fotogramas con los puntos de interés
  landmark_frames = []
  for frame_number in range(frame_count):
    coords = []
    frame = (frames[frame_number] * 255).astype(np.uint8)
    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    landmarks = landmark_predictions[frame_number]
    for channel in range(landmarks.shape[2]):
      landmark = (landmarks[:,:,channel] >= np.max(landmarks[:,:,channel]))
      landmark = np.unravel_index(np.argmax(landmark), landmark.shape)
      landmark = (landmark[1], landmark[0])
      coords.append(landmark)
      cv2.circle(frame, landmark, 2, 255, -1)
    coords = np.array(coords).astype(np.int32).reshape((-1, 1, 2))
    mask = np.zeros_like(mask).astype(np.int32)
    cv2.fillPoly(mask, [cv2.convexHull(coords)], (255))
    frame[:,:,2] = cv2.addWeighted(mask.astype(np.uint8), 0.75, frame[:,:,2], 1, 0)
    cv2.putText(frame, ('L1 ' + video[:-4]), (1,img_height-3), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (255, 255, 255), 1)
    landmark_frames.append(frame)

  # Guardar los fotogramas con las predicciones como GIFs
  imageio.mimsave(os.path.join(gifs_directory, (video[:-4] + '_mask.gif')), mask_frames, fps=fps)
  imageio.mimsave(os.path.join(gifs_directory, (video[:-4] + '_landmarks.gif')), landmark_frames, fps=fps)